<a href="https://colab.research.google.com/github/tyugv/text_clusterization/blob/main/Text_clusterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy2
!pip install pyLDAvis

In [ ]:
import nltk
import gensim
import pandas as pd
import re
import pymorphy2
import os
import pickle
import pyLDAvis
from gensim.models.phrases import Phrases
from pyLDAvis import gensim_models
from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=UserWarning)

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
class Pipeline:
  def __init__(self, stopwords = None):
    if not stopwords:
      stopwords = nltk.corpus.stopwords.words('russian')

    self.stopwords = stopwords
    self.morph = pymorphy2.MorphAnalyzer()

    pyLDAvis.enable_notebook()
    if 'ldavis_prepared' not in os.listdir():
      os.mkdir('ldavis_prepared')
    self.LDAvis_data_filepath = os.path.join('ldavis_prepared', 'ldavis_prepared')
  
  def preprocess(self, text: str):
    text = re.sub("\d+", "", text)
    tokenized = nltk.word_tokenize(text, language="russian")
    preprocessed = []
    for word in tokenized:
      word = self.morph.parse(re.sub(r'[^\w]', '', word))[0].normal_form
      if word != '' and word not in self.stopwords:
        preprocessed.append(word)
    return preprocessed

  @staticmethod
  def generate_N_grams(reviews, N = 1, min_count = 1):
    if N > 1:
      reviews_ngram = reviews.copy()
      for _ in range(N-1):
        phrases = Phrases(reviews_ngram, min_count=min_count)
        reviews_ngram = [phrases[review] for review in reviews_ngram]
      only_ngrams = [[token for token in review  if '_' in token] for review in reviews_ngram]
      return only_ngrams
    return reviews

  @staticmethod
  def make_dictionary(reviews: list):
      return gensim.corpora.Dictionary(reviews)

  @staticmethod
  def make_corpus(dictionary, reviews: list):
      return [dictionary.doc2bow(text) for text in reviews]

  @staticmethod
  def lda_model(corpus, dictionary, num_topics: int = 10):
    return gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)

  def make_vizualization(self, corpus, dictionary, lda_model, filename=None):
    filename = filename or self.LDAvis_data_filepath
    LDAvis_prepared = gensim_models.prepare(lda_model, corpus, dictionary)
    with open(filename, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

  def run_vizualization(self, filename=None):
    filename = filename or self.LDAvis_data_filepath
    with open(filename, 'rb') as f:
      LDAvis_prepared = pickle.load(f)
    pyLDAvis.save_html(LDAvis_prepared, filename +'.html')
    return LDAvis_prepared

  def run(self, reviews_list, n: int = 3, num_topics: int = 10, filename=None):
    filename = filename or self.LDAvis_data_filepath
    reviews = []
    print('preprocessing')
    for review in tqdm_notebook(reviews_list):
      reviews.append(self.preprocess(review))

    ngrams = self.generate_N_grams(reviews, n)
    dictionary = self.make_dictionary(ngrams)
    corpus = self.make_corpus(dictionary, ngrams)
    lda_model = self.lda_model(corpus, dictionary, num_topics=num_topics)

    self.make_vizualization(corpus, dictionary, lda_model, filename)
    return reviews, ngrams, dictionary, corpus, lda_model, self.run_vizualization(filename)

In [ ]:
df = pd.read_csv('<path>', sep=';')

In [ ]:
df.head()

In [ ]:
good_reviews = df[df.rating_value	> 3].review_body
bad_reviews = df[df.rating_value	<= 3].review_body
unknown_reviews = df[df.rating_value.isnull()].review_body

In [ ]:
print(len(good_reviews), len(bad_reviews), len(unknown_reviews))

524 7004 6442


In [ ]:
stopwords = nltk.corpus.stopwords.words('russian')[:46] + ['все', 'еще', 'это', 'год', 'г']
stopwords.remove('не')
stopwords.remove('нет')
pipeline = Pipeline(stopwords)

In [ ]:
_, good_reviews_ngrams, _, _, good_reviews_model, good_reviews_vizualization = \
pipeline.run(good_reviews, filename='good_reviews')

good_reviews_vizualization

preprocessing


  0%|          | 0/524 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.149647  0.041991       1        1  13.593298
1     -0.070505  0.039178       2        1  13.301464
8     -0.050736  0.080566       3        1  12.426714
5     -0.031486  0.017341       4        1  11.098519
7      0.014153 -0.131612       5        1  11.061694
3     -0.041096 -0.031990       6        1   9.699229
2      0.040543 -0.014699       7        1   7.504045
4     -0.033202 -0.013396       8        1   7.351972
0     -0.006563 -0.040271       9        1   7.226513
9      0.029246  0.052892      10        1   6.736552, topic_info=                   Term       Freq      Total Category  logprob  loglift
37    кредитный_договор  19.000000  19.000000  Default  30.0000  30.0000
12              не_мочь  56.000000  56.000000  Default  29.0000  29.0000
1471           один_раз   6.000000   6.000000  Default  28.0000  28.0000
109        без_проблема  11.000000  11.000000  Default  27.0000  27.0000
3038       обработка_пд   3.000000   3.000000  Default  26.0000  26.0000
...                 ...        ...        ...      ...      ...      ...
1520  отдельный_спасибо   1.455696   7.354563  Topic10  -6.1399   1.0778
1634      всякий_случай   1.455731   7.428957  Topic10  -6.1399   1.0677
667           ни_раз_не   1.455715   8.639615  Topic10  -6.1399   0.9168
239        брать_кредит   1.455718   9.459118  Topic10  -6.1399   0.8261
302     являться_клиент   1.455712  17.738474  Topic10  -6.1399   0.1974

[681 rows x 6 columns], token_table=      Topic      Freq                           Term
term                                                
2215      1  0.855950                        _______
850       2  0.357773            акция_привести_друг
850       7  0.357773            акция_привести_друг
3516      7  0.744702                  александр_вна
2575      5  0.850153                алина_григорьев
...     ...       ...                            ...
302       8  0.056375                являться_клиент
302       9  0.056375                являться_клиент
302      10  0.056375                являться_клиент
1154      3  0.880138  являться_клиент_уже_несколько
730       2  0.672391                 являться_очень

[1325 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 2, 9, 6, 8, 4, 3, 5, 1, 10])

In [ ]:
_, bad_reviews_ngrams, _, _, bad_reviews_model, bad_reviews_vizualization = \
pipeline.run(bad_reviews, num_topics=5, filename='bad_reviews')
bad_reviews_vizualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.059478 -0.008678       1        1  22.106129
3      0.072240 -0.016292       2        1  21.653896
1      0.018893  0.075803       3        1  20.060659
0     -0.053358 -0.106682       4        1  19.478070
2     -0.097253  0.055849       5        1  16.701245, topic_info=                         Term         Freq  ...  logprob  loglift
6             кредитный_карта  1493.000000  ...  30.0000  30.0000
3054          мобильный_агент   237.000000  ...  29.0000  29.0000
250               добрый_день   546.000000  ...  28.0000  28.0000
3666          кредит_наличные   356.000000  ...  27.0000  27.0000
597         кредитный_договор   763.000000  ...  26.0000  26.0000
...                       ...          ...  ...      ...      ...
947               такой_образ    34.069655  ...  -6.7722   0.1277
7     позвонить_горячий_линия    38.845674  ...  -6.6410  -0.4904
213                   ещё_раз    38.693897  ...  -6.6449  -0.5406
609                через_день    34.082570  ...  -6.7718  -0.0986
2654        процентный_ставка    35.632818  ...  -6.7273  -0.6398

[415 rows x 6 columns], token_table=       Topic      Freq                        Term
term                                              
6439       3  0.957015               анкетный_дать
14218      1  0.972635             арбитражный_суд
3492       2  0.940950                 без_бейджик
11074      3  0.067175                без_задержка
11074      4  0.940445                без_задержка
...      ...       ...                         ...
142        3  0.124165             являться_клиент
142        4  0.248330             являться_клиент
142        5  0.223497             являться_клиент
1367       4  0.958529     являться_клиент_никогда
3866       5  0.951172  являться_постоянный_клиент

[801 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 1, 3])

In [ ]:
_, unknown_reviews_ngrams, _, _, unknown_reviews_model, unknown_reviews_vizualization = \
pipeline.run(unknown_reviews, num_topics=5, filename='unknown_reviews')
unknown_reviews_vizualization

preprocessing


  0%|          | 0/6442 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.024585 -0.054551       1        1  26.057545
4      0.129085 -0.014473       2        1  21.960893
0     -0.084205 -0.077238       3        1  18.803379
3     -0.056855  0.032771       4        1  17.927314
2     -0.012611  0.113490       5        1  15.250870, topic_info=                         Term        Freq  ...  logprob  loglift
1498        мой_номер_телефон  427.000000  ...  30.0000  30.0000
310             номер_телефон  536.000000  ...  29.0000  29.0000
1309         поступать_звонок  275.000000  ...  28.0000  28.0000
402           человек_который  291.000000  ...  27.0000  27.0000
4772    просить_исключить_мой   93.000000  ...  26.0000  26.0000
...                       ...         ...  ...      ...      ...
314   позвонить_горячий_линия   30.961591  ...  -6.6261  -0.1740
1498        мой_номер_телефон   32.999124  ...  -6.5624  -0.6800
1481         служба_взыскание   29.784732  ...  -6.6648   0.1003
442               третий_лицо   29.629669  ...  -6.6701   0.1316
533                прийти_смс   29.653842  ...  -6.6693   0.0595

[407 rows x 6 columns], token_table=       Topic      Freq                                           Term
term                                                                 
1548       2  0.960673                                     call_центр
7602       3  0.103749  https_wwwbankiruservicesresponsesbankresponse
7602       4  0.881868  https_wwwbankiruservicesresponsesbankresponse
2874       1  0.942266                              агентский_договор
13845      5  0.898523                                более_раз_сутки
...      ...       ...                                            ...
2262       5  0.174471                            являться_клиент_ваш
11240      2  0.718132                               являться_никогда
11240      3  0.164572                               являться_никогда
11240      4  0.029922                               являться_никогда
11240      5  0.074805                               являться_никогда

[793 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 1, 4, 3])

# Результаты

In [ ]:
import textwrap

def print_review(review):
  print('\n'.join(textwrap.wrap(review, 100)))

def reviews_with_keyword(reviews, ngrams, keyword):
  reviews_with_keyword = []
  for i in range(len(reviews)):
    if keyword in ngrams[i]:
      reviews_with_keyword.append(reviews[i])

  print(f'Found {len(reviews_with_keyword)} reviews with keyword {keyword}')
  print('---------')
  for n, review in enumerate(reviews_with_keyword):
    print_review(review)
    print()
    if n == 10:
      break